In [1]:
#-----Importa los paquetes esenciales para correr la simulación.

from __future__ import division
import hoomd
import hoomd.md

In [2]:
#-----Define important variables

press = 3.3;
temp = 9.0;
steps_ramp = 1e3;
run_time = steps_ramp;

In [3]:
#-----Declare the file from wich we'll extract the configuration of the system.

init_file = "T_CM&NP_" + str(temp) + "_P_" + str(press) + "_ramp.gsd"

In [4]:
#-----Start from a gsd file

hoomd.context.initialize("--mode=cpu", memory_traceback = True);

#-----Select the trayectory of the center of mass of particles

snap = hoomd.data.gsd_snapshot(init_file, frame = -1);
system = hoomd.init.read_snapshot(snap);

HOOMD-blue 2.7.0 DOUBLE HPMC_MIXED TBB SSE SSE2 SSE3 
Compiled: 10/02/2019
Copyright (c) 2009-2019 The Regents of the University of Michigan.
-----
You are using HOOMD-blue. Please cite the following:
* J A Anderson, C D Lorenz, and A Travesset. "General purpose molecular dynamics
  simulations fully implemented on graphics processing units", Journal of
  Computational Physics 227 (2008) 5342--5359
* J Glaser, T D Nguyen, J A Anderson, P Liu, F Spiga, J A Millan, D C Morse, and
  S C Glotzer. "Strong scaling of general-purpose molecular dynamics simulations
  on GPUs", Computer Physics Communications 192 (2015) 97--107
-----
HOOMD-blue is running on the CPU
notice(2): Group "all" created containing 1010 particles


In [5]:
meta = system.get_metadata()

In [9]:
f = open('metadata.log', 'a+')
f.write(str(meta))
f.close()

In [10]:
type(meta)

collections.OrderedDict

In [17]:
system.get_metadata()

OrderedDict([('box', <hoomd.data.boxdim at 0x7f9c66baf5c0>),
             ('particles', <hoomd.data.particle_data at 0x7f9c6802bdd8>),
             ('number_density', 0.05846496362752619),
             ('bonds', <hoomd.data.bond_data at 0x7f9c6802be48>),
             ('angles', <hoomd.data.angle_data at 0x7f9c6802beb8>),
             ('dihedrals', <hoomd.data.dihedral_data at 0x7f9c6802bf28>),
             ('impropers', <hoomd.data.dihedral_data at 0x7f9c6802bf98>),
             ('constraints', <hoomd.data.constraint_data at 0x7f9c67f6c048>),
             ('pairs', <hoomd.data.bond_data at 0x7f9c67f6c0b8>),
             ('timestep', 0)])

In [18]:
print(meta)

OrderedDict([('box', <hoomd.data.boxdim object at 0x7f9c6703d6d8>), ('particles', <hoomd.data.particle_data object at 0x7f9c6802bdd8>), ('number_density', 0.05846496362752619), ('bonds', <hoomd.data.bond_data object at 0x7f9c6802be48>), ('angles', <hoomd.data.angle_data object at 0x7f9c6802beb8>), ('dihedrals', <hoomd.data.dihedral_data object at 0x7f9c6802bf28>), ('impropers', <hoomd.data.dihedral_data object at 0x7f9c6802bf98>), ('constraints', <hoomd.data.constraint_data object at 0x7f9c67f6c048>), ('pairs', <hoomd.data.bond_data object at 0x7f9c67f6c0b8>), ('timestep', 0)])


In [5]:
tags = []
for p in system.particles:
    if p.type == 'NP':
        tags.append(p.tag)
for t in tags:
    system.particles.remove(t)

In [6]:
print(tags)

[1000, 1001, 1002, 1003, 1004, 1005, 1006, 1007, 1008, 1009]


In [7]:
#-----Define each rigid body type in the local coordinate system of the body.

rigid = hoomd.md.constrain.rigid();
rigid.set_param('M', 
                types=['A']*8,
                positions=[(-4,0,0),(-3,0,0),(-2,0,0),(-1,0,0),
                           (1,0,0),(2,0,0),(3,0,0),(4,0,0)]);

#-----Instruct the rigid constraint to create the constituent particles

rigid.create_bodies();

#-----Define the potential energy

nl = hoomd.md.nlist.tree()

lj = hoomd.md.pair.lj(r_cut = 3.5, nlist=nl)
lj.set_params(mode='shift')

#-----Define interaction with species in the binary mixture

lj.pair_coeff.set('NP','NP', epsilon = 1.0, sigma = 5.0);
lj.pair_coeff.set('M' ,'M' , epsilon = 1.0, sigma = 1.0);
lj.pair_coeff.set('A' ,'A' , epsilon = 1.0, sigma = 1.0);
lj.pair_coeff.set('M' ,'A' , epsilon = 1.0, sigma = 1.0);
lj.pair_coeff.set('NP','M' , epsilon = 1.0, sigma = 3.0);
lj.pair_coeff.set('NP','A' , epsilon = 1.0, sigma = 3.0);

#-----Select an standard integrator.

hoomd.md.integrate.mode_standard(dt = 0.005);

#------Define some groups and make their union.

nanoparticles = hoomd.group.type(name = 'Nano_Particles', type = 'NP');
mesogens = hoomd.group.rigid_center();
groupNP_mes = hoomd.group.union(name = "NP_Mes", a = nanoparticles, b = mesogens);

#-----Integrate using NPT

#pressure = hoomd.variant.linear_interp(points = [(0,p_prev), (steps_ramp, p_new)])
npt = hoomd.md.integrate.npt(group = groupNP_mes, kT = temp, tau = 5.0, tauP = 0.5, P = press);
npt.randomize_velocities(seed = 42)

#-----Define los archivos en los que se guardará la información de la simulación. 

log_file = "T_" + str(temp) + "_P_" + str(press) + "_noNP.log"
gsd_file = "T_" + str(temp) + "_P_" + str(press) + "_npNP.gsd" 
meso_gsd_file = "T_CM&NP_" + str(temp) + "_P_" + str(press) + "_noNP.gsd"

log = hoomd.analyze.log(filename = log_file,
                         quantities = ['num_particles',
                                     'ndof',
                                     'translational_ndof',
                                     'rotational_ndof',
                                     'potential_energy',
                                     'kinetic_energy',
                                     'translational_kinetic_energy',
                                     'rotational_kinetic_energy',
                                     'temperature',
                                     'pressure',
                                      'volume'],
                         period = 1e1,
                         overwrite = True);
gsd = hoomd.dump.gsd(gsd_file,
               period = 1e1,
               group = hoomd.group.all(),
               overwrite = True); 
meso_gsd = hoomd.dump.gsd(meso_gsd_file,
               period = 1e1,
               group = groupNP_mes,
               overwrite = True);

notice(2): constrain.rigid(): Creating 1000 rigid bodies (adding 8000 particles)
-----
You are using tree neighbor lists. Please cite the following:
* M P Howard, J A Anderson, A Nikoubashman, S C Glotzer, and A Z
  Panagiotopoulos. "Efficient neighbor list calculation for molecular simulation
  of colloidal systems using graphics processing units", Computer Physics
  Communications 203 (2016) 45--52
-----
notice(2): Group "Nano_Particles" created containing 0 particles
notice(2): Group "rigid_center" created containing 1000 particles
notice(2): Group "NP_Mes" created containing 1000 particles


In [8]:
#-----Run the simulation.

hoomd.run(run_time)

notice(2): -- Neighborlist exclusion statistics -- :
notice(2): Particles with 0 exclusions             : 9000
notice(2): Neighbors included by diameter          : no
notice(2): Neighbors excluded when in the same body: yes
** starting run **
Time 00:00:10 | Step 211 / 1000 | TPS 20.9855 | ETA 00:00:37
Time 00:00:20 | Step 421 / 1000 | TPS 20.9798 | ETA 00:00:27
Time 00:00:30 | Step 630 / 1000 | TPS 20.7772 | ETA 00:00:17
Time 00:00:40 | Step 843 / 1000 | TPS 21.2421 | ETA 00:00:07
Time 00:00:47 | Step 1000 / 1000 | TPS 21.0916 | ETA 00:00:00
Average TPS: 21.0107
---------
-- Neighborlist stats:
490 normal updates / 10 forced updates / 0 dangerous updates
n_neigh_min: 0 / n_neigh_max: 135 / n_neigh_avg: 60.2838
shortest rebuild period: 1
notice(2): Total amount of managed memory allocated through Global[Array,Vector]: 0B
notice(2): Actual allocation sizes may be larger by up to the OS page size due to alignment.
notice(2): List of memory allocations and last 3 functions called at time 